In [ ]:
%matplotlib inline
import os, sys; sys.path.append(os.path.expanduser(os.getcwd()))
from dataloader.pytorch_lightning_datamodule import OmnidataDataModule

# You can install the libs with a clean Conda install using:
# conda create -n omnidata python=3.8
# conda install pytorch=1.12.0 torchvision torchaudio pytorch-cuda=11.6 -c pytorch -c nvidia
# conda install -c fvcore -c iopath -c conda-forge fvcore iopath
# conda install pytorch3d -c pytorch3d
# conda install plotly fastcore


# Multiview (all views fixated on same point)

In [ ]:
os.environ['OMNIDATA_PATH'] = '/scratch/data/omnidata'
os.environ['OMNIDATA_CACHE_PATH'] = '/scratch/data/omnidata/.cache'

dm = OmnidataDataModule(
       tasks = ['point_info', 'rgb', 'depth_euclidean', 'normal', 'mask_valid'],
      train_datasets_to_options = dict(
            # TaskonomyDataset = dict(data_amount='tiny', data_path='/datasets/omnidata/ta/'),
            ReplicaDataset   = dict()
        ),
        eval_datasets_to_options  = dict(
            # HypersimDataset   = dict(cooccurrence_method='FRAGMENTS')
            ReplicaDataset   = dict()
            # GSOReplicaDataset   = dict()
            # TaskonomyDataset  = dict(data_amount='tiny')
        ),
        shared_options = dict(
            data_amount  = 'debug',
            data_path    = os.environ['OMNIDATA_PATH'],
            cache_dir    = os.environ['OMNIDATA_CACHE_PATH'],
            image_size   = 512,
            multiview_sampling_method = 'FILENAME', # Works for Taskonomy, Replica
            n_workers    = None,
            # force_refresh_tmp = True,
            num_positive = 3,
        ),
        train_options  = dict(),
        eval_options   = dict(),
        dataloader_kwargs = dict(batch_size=4),
)
dm.setup(stage='val')


In [ ]:
# Load a batch (3 views fixated on the same point)
dl = dm.val_dataloader()[0]
for batch in dl:
    batch = batch['positive']
    break

### Visualization

In [ ]:
from dataloader.pytorch3d_utils import *
from dataloader.viz_utils import *

# Show the RGB image
show_batch_images(batch, batch_idx=0, view_idxs=[0,1,2], keys=['rgb'])


In [ ]:
# Show all unprojected pointclouds in the same scene, and draw wireframes for the cameras
show_batch_scene(batch, batch_idx=0, view_idxs=[0,1,2])


# Multiview (nonfixated, but overlapping)

In [ ]:
# You may need to download the scene structs
# Put them in the same folder as the rest of the Omnidata dataset
# So it should be like:
# path_to_omnidata/
#   point_info/
#   rgb/
#   ...
#   scene_metadata/
#   scene_multiview_metadata/
# It will work for Taskonomy, Replica, Hypersim, and GSO-in-Replica
# How to download:
# pip install gdown
# gdown '1bvdgtHtKHEtSwYpYlNz2bIxilvinfC54'
# gdown '1avTBx5JbKj9GiGk9rWsufGppo1DxqQ-l'
# tar -xf scene_metadata_hs_r_gso_t.tar
# tar -xf scene_multiview_metadata_hs_r_gso_t.tar
# 

In [ ]:
os.environ['OMNIDATA_PATH'] = '/scratch/data/omnidata'
os.environ['OMNIDATA_CACHE_PATH'] = '/scratch/data/omnidata/.cache'

dm = OmnidataDataModule(
       tasks = ['point_info', 'rgb', 'depth_euclidean', 'normal', 'mask_valid'],
      train_datasets_to_options = dict(
            # TaskonomyDataset = dict(data_amount='tiny', data_path='/datasets/omnidata/ta/'),
            ReplicaDataset   = dict()
        ),
        eval_datasets_to_options  = dict(
            # HypersimDataset   = dict(cooccurrence_method='FRAGMENTS')
            ReplicaDataset   = dict()
            # GSOReplicaDataset   = dict()
            # TaskonomyDataset  = dict(data_amount='tiny')
        ),
        shared_options = dict(
            data_amount  = 'debug',
            data_path    = os.environ['OMNIDATA_PATH'],
            cache_dir    = os.environ['OMNIDATA_CACHE_PATH'],
            image_size   = 512,
            n_workers    = None,
            multiview_sampling_method = 'CENTER_VISIBLE',
            # force_refresh_tmp = True,
            num_positive = 3,
        ),
        train_options  = dict(),
        eval_options   = dict(),
        dataloader_kwargs = dict(batch_size=4),
)
dm.setup(stage='val')


In [ ]:
# Show the RGB image
show_batch_images(batch, batch_idx=0, view_idxs=[0,1,2], keys=['rgb'])

# Show all unprojected pointclouds in the same scene, and draw wireframes for the cameras
show_batch_scene(batch, batch_idx=0, view_idxs=[0,1,2])
